In [1]:
import pandas as pd
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LinearRegression

In [2]:
def rfefeature(indep_x,dep_y,n):
    rfelist=[]
    lin=LinearRegression()
    RF=RandomForestRegressor(n_estimators = 10, random_state = 0)
    DT=DecisionTreeRegressor(random_state = 0)
    svc_model =SVR(kernel = 'linear')
    
    rfemodellist=[lin,RF,DT,svc_model]
    
    for i in rfemodellist:
        log_rfe = RFE(estimator=i, n_features_to_select=n)
        log_fit = log_rfe.fit(indep_x,dep_y)
        log_rfe_feature=log_fit.transform(indep_x)
        rfelist.append(log_rfe_feature)
    return rfelist

def split_scalar(indep_x,dep_y):
    x_train,x_test,y_train,y_test=train_test_split(indep_x,dep_y,test_size=0.25,random_state=0)
    sc=StandardScaler()
    x_train=sc.fit_transform(x_train)
    x_test=sc.transform(x_test)
    return x_train,x_test,y_train,y_test

def R2_prediction(regressor,x_test,y_test):
    y_pred=regressor.predict(x_test)
    from sklearn.metrics import r2_score
    R2=r2_score(y_test,y_pred)
    return R2

def linear(x_train,y_train,x_test):
    from sklearn.linear_model import LinearRegression
    regressor=LinearRegression()
    regressor.fit(x_train,y_train)
    R2=R2_prediction(regressor,x_test,y_test)
    return R2

def svm_linear(x_train,y_train,x_test):
    regressor=SVR(kernel="linear")
    regressor.fit(x_train,y_train)
    R2=R2_prediction(regressor,x_test,y_test)
    return R2

def svm_NL(x_train,y_train,x_test):
    regressor=SVR(kernel="rbf")
    regressor.fit(x_train,y_train)
    R2=R2_prediction(regressor,x_test,y_test)
    return R2

def DecisionTree(x_train,y_train,x_test):
    from sklearn.tree import DecisionTreeRegressor
    regressor=DecisionTreeRegressor(random_state=0)
    regressor.fit(x_train,y_train)
    R2=R2_prediction(regressor,x_test,y_test)
    return R2

def RandomForest(x_train,y_train,x_test):
    from sklearn.ensemble import RandomForestRegressor
    regressor=RandomForestRegressor(n_estimators=10,random_state=0)
    regressor.fit(x_train,y_train)
    R2=R2_prediction(regressor,x_test,y_test)
    return R2

In [3]:
def rfe_Regression(acclin,accsvml,accdes,accrf):
    rfedataframe=pd.DataFrame(index=["Linear","SVC","Random","DecisionTree"],columns=["linear","SVMl","Decision","Random"])
    for number,index in enumerate(rfedataframe.index):
        rfedataframe["linear"][index]=acclin[number]
        rfedataframe["SVMl"][index]=accsvml[number]
        rfedataframe["Decision"][index]=accdes[number]
        rfedataframe["Random"][index]=accrf[number]
    return rfedataframe    

In [4]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df=pd.get_dummies(df2,drop_first=True)
indep_x=df.drop("classification_yes",axis=1)
dep_y=df["classification_yes"]

In [5]:
df2

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [6]:
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [7]:
rfelist=rfefeature(indep_x,dep_y,2)
acclin=[]
accsvml=[]
accdes=[]
accrf=[]

In [8]:
rfelist

[array([[1., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0

In [9]:
for i in rfelist:
    x_train,x_test,y_train,y_test=split_scalar(i,dep_y)

    r2_lin=linear(x_train,y_train,x_test)
    acclin.append(r2_lin)

    r2_sl=svm_linear(x_train,y_train,x_test)
    accsvml.append(r2_sl)

    r2_d=DecisionTree(x_train,y_train,x_test)
    accdes.append(r2_d)

    r2_r=RandomForest(x_train,y_train,x_test)
    accrf.append(r2_r)

    result=rfe_Regression(acclin,accsvml,accdes,accrf)


IndexError: list index out of range

In [ ]:
result
#2

In [ ]:
result
#3

In [ ]:
result
#4

In [ ]:
result
#5

In [ ]:
result
#6

In [ ]:
result
#7